## Qwen2.5-0.5B-Instruct 模型下载部署调用标准流程下的OOP封装

In [1]:
import torch
from modelscope import snapshot_download
from modelscope import AutoModelForCausalLM, AutoTokenizer

In [6]:
class LLMChatBot:
    def __init__(self,model_id_or_path = "Qwen/Qwen2.5-0.5B-Instruct",device = "auto"):
        print(f"正在下载模型，加载分词器和模型")
        self.model_dir = snapshot_download(model_id_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_dir,
            trust_remote_code = True
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_dir,
            device_map = device,
            torch_dtype = "auto",
            trust_remote_code = True
        )
        self.history = [
            {'role':'system',"content":"你是一个资深的中医，可以根据临床症状给出医疗诊断和治疗方案"}
        ]
        print(f"模型已经加载完毕，可以进行对话")


    def chat(self,user_input,max_new_tokens = 512, temperature = 0.7):
        self.history.append({"role":'user',"content":user_input})
        text = self.tokenizer.apply_chat_template(
            self.history,
            tokenize = False,
            add_generation_prompt = True
        )
        model_inputs = self.tokenizer([text],return_tensors = 'pt').to(self.model.device)
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens = 512,
            temperature = temperature,
            do_sample = True if temperature > 0 else False 
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids,output_ids in zip(model_inputs.input_ids,generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids,skip_special_tokens = True)[0]
        self.history.append({'role':'assistent',"content":response})

        return response

    def clear_history(self):
        self.history = [self.history[0]]
        print(f"历史对话已经清空，只保留system prompt")

In [7]:
chatbot = LLMChatBot()

正在下载模型，加载分词器和模型


2025-12-30 15:07:29,222 - modelscope - INFO - Creating symbolic link [C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2.5-0.5B-Instruct].
2025-12-30 15:07:29,222 - modelscope - WARNING - Failed to create symbolic link C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2.5-0.5B-Instruct for C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2___5-0___5B-Instruct.


模型已经加载完毕，可以进行对话


In [8]:
ans1 = chatbot.chat("我月经超级疼，出血量少，会有血块，腰疼肚子疼，腰和肚子都特别冷")
print(ans1)

根据您的描述，这可能是痛经（也称为月经痛）的症状。痛经是一种常见的妇科问题，可能由多种因素引起，包括激素变化、子宫内膜异位症等。

1. **病因分析**：
   - 激素变化：女性在月经周期中经历雌激素和孕酮水平的变化，这些变化可能会导致痛经。
   - 子宫内膜异位症：这是一种子宫内部组织生长的情况，可能导致疼痛和其他不适。
   - 肠道问题：如便秘或腹泻也可能影响到腹部的感觉。
   - 其他健康状况：如甲状腺功能异常等。

2. **治疗方法**：
   - 保持良好的生活习惯，如规律作息、适量运动、均衡饮食。
   - 使用热敷或热水袋来缓解下腹部的疼痛感。
   - 避免使用可能加重症状的药物，除非医生建议。
   - 如果怀疑是子宫内膜异位症，可以考虑咨询妇科专家进行进一步检查和治疗。

3. **其他建议**：
   - 尽快就医，寻求专业意见，以获得准确的诊断和个性化的治疗方案。

请注意，以上信息仅供参考，具体治疗应遵循医生的专业指导。希望您早日康复！


In [9]:
ans2 = chatbot.chat("我觉得有可能是前列腺素分泌过多了。是否有方法可以抑制前列腺素的分泌，让我在月经来时不会疼？")

In [10]:
print(ans2)

根据您的描述，您可能患上了子宫内膜异位症或者卵巢囊肿等妇科疾病。这些病症会导致月经痛、腹痛等症状，并且需要通过药物治疗才能缓解疼痛。

建议您尽快就医进行详细检查，以明确病因并采取相应的治疗措施。医生可能会推荐使用一些药物如非甾体抗炎药（NSAIDs）来减轻疼痛和炎症；对于慢性盆腔痛，还可能需要考虑手术治疗。

请注意休息，避免过度劳累，保持良好的生活习惯，有助于身体恢复。同时，定期复查也是非常重要的。


In [14]:
print(chatbot.history)

[{'role': 'system', 'content': '你是一个资深的中医，可以根据临床症状给出医疗诊断和治疗方案'}, {'role': 'user', 'content': '我月经超级疼，出血量少，会有血块，腰疼肚子疼，腰和肚子都特别冷'}, {'role': 'assistent', 'content': ['根据您的描述，这可能是痛经（也称为月经痛）的症状。痛经是一种常见的妇科问题，可能由多种因素引起，包括激素变化、子宫内膜异位症等。\n\n1. **病因分析**：\n   - 激素变化：女性在月经周期中经历雌激素和孕酮水平的变化，这些变化可能会导致痛经。\n   - 子宫内膜异位症：这是一种子宫内部组织生长的情况，可能导致疼痛和其他不适。\n   - 肠道问题：如便秘或腹泻也可能影响到腹部的感觉。\n   - 其他健康状况：如甲状腺功能异常等。\n\n2. **治疗方法**：\n   - 保持良好的生活习惯，如规律作息、适量运动、均衡饮食。\n   - 使用热敷或热水袋来缓解下腹部的疼痛感。\n   - 避免使用可能加重症状的药物，除非医生建议。\n   - 如果怀疑是子宫内膜异位症，可以考虑咨询妇科专家进行进一步检查和治疗。\n\n3. **其他建议**：\n   - 尽快就医，寻求专业意见，以获得准确的诊断和个性化的治疗方案。\n\n请注意，以上信息仅供参考，具体治疗应遵循医生的专业指导。希望您早日康复！']}, {'role': 'user', 'content': '我觉得有可能是前列腺素分泌过多了。是否有方法可以抑制前列腺素的分泌，让我在月经来时不会疼？'}, {'role': 'assistent', 'content': ['根据您的描述，您可能患上了子宫内膜异位症或者卵巢囊肿等妇科疾病。这些病症会导致月经痛、腹痛等症状，并且需要通过药物治疗才能缓解疼痛。\n\n建议您尽快就医进行详细检查，以明确病因并采取相应的治疗措施。医生可能会推荐使用一些药物如非甾体抗炎药（NSAIDs）来减轻疼痛和炎症；对于慢性盆腔痛，还可能需要考虑手术治疗。\n\n请注意休息，避免过度劳累，保持良好的生活习惯，有助于身体恢复。同时，定期复查也是非常重要的。']}]
